In [ ]:
!pip uninstall transformers
!pip install transformers==4.45.2

Found existing installation: transformers 4.47.1
Uninstalling transformers-4.47.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.47.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.47.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0


In [ ]:
pip install datasets torch pandas sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from evaluate import load

data_path = "processed_file.csv"
model_name = "VietAI/vit5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

In [ ]:
# Load dữ liệu từ file CSV
df = pd.read_csv(data_path)
print(f"Dữ liệu sau xử lý: {len(df)} dòng")

# Chuyển dữ liệu thành Dataset
dataset = Dataset.from_pandas(df)

# Tách dữ liệu thành train, validation, test
split_data = dataset.train_test_split(test_size=0.2, seed=42)
temp_split = split_data["test"].train_test_split(test_size=0.5, seed=42)

data_splits = DatasetDict({
    "train": split_data["train"],
    "validation": temp_split["train"],
    "test": temp_split["test"]
})

print(f"Tập huấn luyện: {len(data_splits['train'])}")
print(f"Tập đánh giá: {len(data_splits['validation'])}")
print(f"Tập kiểm tra: {len(data_splits['test'])}")


Dữ liệu sau xử lý: 1350 dòng
Tập huấn luyện: 1080
Tập đánh giá: 135
Tập kiểm tra: 135


In [ ]:
# Hàm Tokenize dữ liệu
def tokenize_function(examples):
    inputs = examples['processed_informal']
    targets = examples['processed_formal']
    model_inputs = tokenizer(inputs, max_length=128, padding="max_length", truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = data_splits.map(tokenize_function, batched=True)

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4109: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

In [ ]:
tokenized_data["train"].to_pandas().to_json("train.json", orient="records", lines=True, force_ascii=False)
tokenized_data["validation"].to_pandas().to_json("validation.json", orient="records", lines=True, force_ascii=False)
tokenized_data["test"].to_pandas().to_json("test.json", orient="records", lines=True, force_ascii=False)

print("Dữ liệu đã được lưu vào các file JSON!")


Dữ liệu đã được lưu vào các file JSON!


In [ ]:
# Đọc các file JSON và chuyển đổi thành DataFrame
train_df = pd.read_json("train.json", orient="records", lines=True)
validation_df = pd.read_json("validation.json", orient="records", lines=True)
test_df = pd.read_json("test.json", orient="records", lines=True)

# Chuyển đổi DataFrame thành Dataset
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

# Tạo DatasetDict từ các Dataset
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

print("Dữ liệu đã được tải lên và chuyển thành DatasetDict!")


Dữ liệu đã được tải lên và chuyển thành DatasetDict!


In [ ]:
from transformers import DataCollatorForSeq2Seq
# Cấu hình huấn luyện
training_args = Seq2SeqTrainingArguments(
    output_dir="tmp/",
    do_train=True,
    do_eval=True,
    num_train_epochs=30,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    logging_dir='./log',
    group_by_length=True,
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=3,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Huấn luyện mô hình với eval
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    data_collator=data_collator
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,6.037000,0.246227
1000,0.241900,0.215907
1500,0.178200,0.219525
2000,0.138400,0.230183
2500,0.112200,0.242456
3000,0.086600,0.263688
3500,0.072400,0.277966
4000,0.059600,0.294267
4500,0.050400,0.308301
5000,0.043500,0.315224


TrainOutput(global_step=8100, training_loss=0.4450453429457582, metrics={'train_runtime': 2577.9458, 'train_samples_per_second': 12.568, 'train_steps_per_second': 3.142, 'total_flos': 4932558913536000.0, 'train_loss': 0.4450453429457582, 'epoch': 30.0})

In [ ]:
# Lưu mô hình vào thư mục tạm thời
trainer.save_model("/content/model")

In [ ]:
from google.colab import files
import shutil
# Nén thư mục mô hình để tải về
shutil.make_archive("/content/model", 'zip', "/content", "model")

# Tải tệp zip về
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>